In [1]:
import pandas as pd
import numpy as np

In [2]:
from tensorflow.python import keras

In [3]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, BatchNormalization

In [4]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [5]:
data_gen = ImageDataGenerator(validation_split = 0.2)
training = data_gen.flow_from_directory('asl-alphabet/asl_alphabet_train/asl_alphabet_train', 
                                         class_mode = 'categorical', 
                                         batch_size = 64,
                                         target_size = (200, 200),
                                         subset = 'training')
validating = data_gen.flow_from_directory('asl-alphabet/asl_alphabet_train/asl_alphabet_train',
                                           target_size = (200, 200),
                                           subset = 'validation')

Found 62400 images belonging to 26 classes.
Found 15600 images belonging to 26 classes.


In [6]:
print (training.class_indices)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}


In [7]:
asl_model = Sequential()

In [8]:
asl_model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(200,200,3)))
asl_model.add(Conv2D(20, kernel_size=(2, 2), activation='relu'))
asl_model.add(Conv2D(20, kernel_size=(3, 3), activation='relu'))
asl_model.add(Conv2D(20, kernel_size=(1, 1), activation='relu'))

asl_model.add(Flatten())
asl_model.add(Dense(128, activation='relu'))
asl_model.add(BatchNormalization())
asl_model.add(Dense(128, activation='relu'))
asl_model.add(Dense(26, activation='softmax'))

In [9]:
asl_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 197, 197, 20)      5140      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 195, 195, 20)      3620      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 195, 195, 20)      420       
_________________________________________________________________
flatten (Flatten)            (None, 760500)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               97344128  
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               5

In [10]:
asl_model.compile(loss = 'categorical_crossentropy',
                  optimizer = 'adam',
                  metrics = (['accuracy']))

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
asl_model.fit(training, 
              epochs=15, 
              validation_data=validating,
              callbacks=[EarlyStopping(patience=3, restore_best_weights=True),
                         ReduceLROnPlateau(patience=2)])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 975 steps, validate for 488 steps
Epoch 1/15
975/975 [==============================] - 4533s 5s/step - loss: 0.8554 - accuracy: 0.7421 - val_loss: 2.5117 - val_accuracy: 0.4367
Epoch 2/15
975/975 [==============================] - 4474s 5s/step - loss: 0.0895 - accuracy: 0.9739 - val_loss: 3.5482 - val_accuracy: 0.3726
Epoch 3/15
975/975 [==============================] - 4469s 5s/step - loss: 0.0408 - accuracy: 0.9881 - val_loss: 2.6421 - val_accuracy: 0.5147
Epoch 4/15
975/975 [==============================] - 4492s 5s/step - loss: 0.0124 - accuracy: 0.9985 - val_loss: 1.2246 - val_accuracy: 0.6945
Epoch 5/15
975/975 [==============================] - 4717s 5s/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 1.3932 - val_accuracy: 0.6824
Epoch 6/15
975/975 [==============================] - 5340s 5s/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 1.5436 - val_accuracy: 0.6744
Epoch 7/15
975/975 [======================

In [13]:
asl_model.save('200p-model.h5')